# Chapter 5: Recurrent Neural Network

## 5.3 Implementing RNNs

### 5.3.1 Implementing RNN Layer

Forward propagation of RNN layer is given by the following formula.
$$
    h_t = tanh(h_{t-1} W_h + x_{t} W_{x} + b)
$$

Matrix size of $h_{t-1}$ and $W_{h}$ are `$(N, H)$` and `(H, H)` respectively.
Also $x_{t}$ and $W_{x}$ are `(N, D)` and `(D, H)`, then $h_t$ is `(N, H)`.

Note that, N is number of samples, H is hidden size, D is input size.

In [1]:
import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next
